<a href="https://colab.research.google.com/github/TaiseiYamana/optuna_study/blob/main/optuna_tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optuna チュートリアル1

## 参考本
https://www.ohmsha.co.jp/book/9784274230103/
## GitHub
https://github.com/pfnet-research/optuna-book

In [ ]:
# Optunaのインストール
!pip3 install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.5 MB/s eta 0:00:00


# 簡単な数式の最適化

Optuna を使って以下の数式$f$の最適化をしましょう。
<br />
<br />
$f(x,y) = (1.5 - x + xy)^2 + (2.25 -x +xy^2)^2 + (2.625 -x + xy^3)^2 $

$(-4.5\le x \le, -4.5 \le y \le 4.5)$

$f$を最小化する最適解 : $( x, y) = (3,0.5)$
<br />
<br />

Optunaでは以下の 4 つのステップを実装していきます。
1.   目的関数を定義する
2.   目的関数の内部で適当な変数を決める ***(suggest)***
3.   実験 ***study*** オブジェクトを作成する。
4.   施行 ***(trial)*** の回数を設定して最適化を開始する ***(optimize)***:

In [ ]:
# Optunaをインポート
import optuna

# 目的関数を定義
def objective(trial): # 引数に　trial を指定
    # サジェストAPIで目的関数への入力を定案
    x = trial.suggest_float("x", -4.5, 4.5)
    y = trial.suggest_float("y", -4.5, 4.5)

    # 目的関数の評価値をreturn
    return (1.5 - x + x * y) ** 2 + \
        (2.25 - x + x * y ** 2) ** 2 + \
        (2.625 - x + x * y ** 3) ** 2

In [ ]:
# 最適化を実行
study = optuna.create_study(direction="minimize") # directionで最小化を指定
study.optimize(objective, n_trials=1000)

# 最適化の結果を確認
print(f"Best objective value: {study.best_value}")
print(f"Best parameter: {study.best_params}")

[I 2023-06-28 15:14:16,650] A new study created in memory with name: no-name-a19e56cd-ad46-4a5d-97e3-6e6726592497
[I 2023-06-28 15:14:16,656] Trial 0 finished with value: 12092.438807663988 and parameters: {'x': 1.782169066320792, 'y': 3.896278834478153}. Best is trial 0 with value: 12092.438807663988.
[I 2023-06-28 15:14:16,659] Trial 1 finished with value: 113.65300772624997 and parameters: {'x': -0.31543467217633214, 'y': -2.858595138169935}. Best is trial 1 with value: 113.65300772624997.
[I 2023-06-28 15:14:16,664] Trial 2 finished with value: 42013.40155072768 and parameters: {'x': 2.2492362229336633, 'y': 4.459230943675033}. Best is trial 1 with value: 113.65300772624997.
[I 2023-06-28 15:14:16,669] Trial 3 finished with value: 0.05951176539178722 and parameters: {'x': 3.284772464823055, 'y': 0.6002107256796583}. Best is trial 3 with value: 0.05951176539178722.
[I 2023-06-28 15:14:16,673] Trial 4 finished with value: 2295.878246996959 and parameters: {'x': 0.9769295849617956, 'y

Best objective value: 0.0009618638723908427
Best parameter: {'x': 2.959260257308685, 'y': 0.495300225564798}


# Studyの保存と再開
Studyはローカルファイルとして保存することが可能です。Studyを保存することで以下のメリットがあります。
## メリット
- 長時間にわたる最適化の途中経過を調べることができる。
- 何らかの原因で最適化が中断された場合に、途中から再開できる。
- 可視化機能などを使い、最適化の傾向を後から分析できる。

In [ ]:
from optuna.study import MaxTrialsCallback

# セーブ先の設定を含めた最適化の実行
storage="sqlite:///optuna.db"
study_name="function_minimize"
study = optuna.create_study(
    direction="minimize",
    study_name=study_name,
    storage=storage, # 保存先を"sqlite:///"から始める文字列を指定
    load_if_exists=True #　既存のstudy_nameのstudyは作成エラーになるため、load_if_exitsをTrueにする。
)

n_trials = 100
study.optimize(objective,
               n_trials=n_trials,
               callbacks=[MaxTrialsCallback(n_trials)])  # MaxTrialsCalbackで再開後の終了trialsを指定

[I 2023-07-02 15:01:33,108] Using an existing study with name 'function_minimize' instead of creating a new one.
[I 2023-07-02 15:01:33,252] Trial 55 finished with value: 14.02432431971972 and parameters: {'x': 3.9687944785349747, 'y': -0.15651512525455474}. Best is trial 29 with value: 0.027249374122802338.
[I 2023-07-02 15:01:33,366] Trial 56 finished with value: 40.26600225783084 and parameters: {'x': 3.57677255307173, 'y': 1.171218633356017}. Best is trial 29 with value: 0.027249374122802338.
[I 2023-07-02 15:01:33,479] Trial 57 finished with value: 200.75663027875908 and parameters: {'x': 2.821380052749278, 'y': 1.6263797008293264}. Best is trial 29 with value: 0.027249374122802338.
[I 2023-07-02 15:01:33,603] Trial 58 finished with value: 0.5062147803757973 and parameters: {'x': 2.681420653375141, 'y': 0.20392405623640375}. Best is trial 29 with value: 0.027249374122802338.
[I 2023-07-02 15:01:33,705] Trial 59 finished with value: 0.48662479923709345 and parameters: {'x': 3.22120

In [ ]:
# Studyのロード
study = optuna.load_study(
    storage=storage,
    study_name=study_name,
)
study.optimize(objective, n_trials=100)

[I 2023-07-02 15:06:29,801] Trial 100 finished with value: 0.09712279700659136 and parameters: {'x': 3.2842807230627313, 'y': 0.6117548319950098}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 15:06:29,912] Trial 101 finished with value: 0.0921311115979917 and parameters: {'x': 3.5872490543176783, 'y': 0.5779949595899861}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 15:06:30,025] Trial 102 finished with value: 0.467064037567981 and parameters: {'x': 3.330722223054503, 'y': 0.4452664496009074}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 15:06:30,133] Trial 103 finished with value: 0.12102922427658372 and parameters: {'x': 3.1970610693637536, 'y': 0.6041638479588184}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 15:06:30,252] Trial 104 finished with value: 1.7300289317729285 and parameters: {'x': 2.460687556321671, 'y': -0.13597299685584768}. Best is trial 91 with value: 0.015209582269158257.
[I 2023-07-02 

KeyboardInterrupt: ignored

## 探索結果の可視化
最適化の探索結果を可視化することができます。
Optunaの可視化はoptuna.visualizationメソッドを用います。

公式ドキュメント:   
https://optuna.readthedocs.io/en/stable/reference/visualization/index.html  
公式サンプルコード：  
https://github.com/optuna/optuna-examples/blob/main/visualization/plot_study.ipynb


In [ ]:
# trials_dataframe()より、最適化の履歴をpadasのDataFrameに変換
df = study.trials_dataframe()
df = df.sort_values("value", ascending=True)
print(df[["value"]].head())

      number         value             datetime_start  \
0          0   2423.024304 2023-06-28 16:29:28.471248   
1          1      4.780134 2023-06-28 16:29:28.598956   
2          2     20.366460 2023-06-28 16:29:28.703831   
3          3  21173.386775 2023-06-28 16:29:28.801780   
4          4     78.617213 2023-06-28 16:29:28.905739   
...      ...           ...                        ...   
1060    1060      6.053593 2023-06-28 16:36:28.083203   
1061    1061     35.578543 2023-06-28 16:36:28.235002   
1062    1062     21.050963 2023-06-28 16:36:28.404811   
1063    1063      5.986315 2023-06-28 16:36:28.565746   
1064    1064      0.008361 2023-06-28 16:36:28.718769   

              datetime_complete               duration  params_x  params_y  \
0    2023-06-28 16:29:28.553170 0 days 00:00:00.081922 -2.311581 -2.627256   
1    2023-06-28 16:29:28.668209 0 days 00:00:00.069253  1.799022 -0.743868   
2    2023-06-28 16:29:28.766135 0 days 00:00:00.062304  0.669671  1.251194   
3  